In [ ]:
import pandas as pd
import plotly.express as px
import statistics
from scipy import stats
import numpy as np

In [ ]:
asterName = "Asteroids/Asteroid "

## We add all files to the same data frame
This is to get an overall understanding of the drilling process and relations between variables

In [ ]:
BIT_DEPTH = []
RATE_OF_PENETRATION = []
HOOK_LOAD = []
DIFFERENTIAL_PRESSURE = []
WEIGHT_ON_BIT = []
DRILL_BIT_ID = []
DRILL_BIT_NAME = []
RUN_ID = []
TIME = []
curTime=0.0 #hours
runNumber = 0

When reading in the files we catch bad data using a few methods
1. Nan values
2. Negative Depth
3. Ascending drill
4. Miscalculated rates of penetration

In [ ]:
for i in range(1, 21):
    prevName=""
    curTime=0
    prevPrevBitDepth=-1.0
    prevBitDepth=-1.0
    currName = asterName + str(i) + ".csv"
    df = pd.read_csv(currName)
    for index, row in df.iterrows():
        if(row['BIT_DEPTH'] != row['BIT_DEPTH'] or row['RATE_OF_PENETRATION'] != row['RATE_OF_PENETRATION'] or row['HOOK_LOAD'] != row['HOOK_LOAD']):
            continue
        if(row['BIT_DEPTH']<=0):
            continue
        if(prevBitDepth!=-1 and row['BIT_DEPTH']-prevBitDepth < 0):
            prevBitDepth=prevPrevBitDepth
            continue
        if(row['RATE_OF_PENETRATION']<8):
            continue
        BIT_DEPTH.append(row['BIT_DEPTH'])
        RATE_OF_PENETRATION.append(row['RATE_OF_PENETRATION'])
        HOOK_LOAD.append(row['HOOK_LOAD'])
        DIFFERENTIAL_PRESSURE.append(row['DIFFERENTIAL_PRESSURE'])
        WEIGHT_ON_BIT.append(row['WEIGHT_ON_BIT'])
        DRILL_BIT_ID.append(row['DRILL_BIT_ID'])
        DRILL_BIT_NAME.append(row['DRILL_BIT_NAME'])
        if prevBitDepth != -1:
            curTime+=(row['BIT_DEPTH']-prevBitDepth)/(row['RATE_OF_PENETRATION']) if row['RATE_OF_PENETRATION']!= 0.0 else 0.0
        prevPrevBitDepth=prevBitDepth
        prevBitDepth=row['BIT_DEPTH']
        TIME.append(curTime)
        if(prevName != row['DRILL_BIT_NAME']):
            runNumber += 1
            prevName = row['DRILL_BIT_NAME']
            prevBitDepth=-1.0
            curTime=0
        RUN_ID.append(runNumber)

In [ ]:
f = pd.DataFrame(columns = ['BIT DEPTH', 'RATE OF PENETRATION', 'HOOK LOAD', 'DIFFERENTIAL PRESSURE', 'WEIGHT ON BIT', 'DRILL BIT ID', 'DRILL BIT NAME', 'RUN ID', 'TIME'])

In [ ]:
f['TIME']=TIME
f['BIT DEPTH'] = BIT_DEPTH
f['RATE OF PENETRATION'] = RATE_OF_PENETRATION
f['HOOK LOAD'] = HOOK_LOAD
f['DIFFERENTIAL PRESSURE'] = DIFFERENTIAL_PRESSURE
f['WEIGHT ON BIT'] = WEIGHT_ON_BIT
f['DRILL BIT ID'] = DRILL_BIT_ID
f['DRILL BIT NAME'] = DRILL_BIT_NAME
f['RUN ID'] = RUN_ID


After reading the data from all the asteroid files, we can now visualise some relations. First we have the graph of Bit Depth to Time, from which we can see the approximate depths that drills slow down at.

In [ ]:
fig = px.scatter(f, x='TIME', y='BIT DEPTH', color='DRILL BIT NAME')
fig.show()

## Read the data for different drills
We have separated the data based on Drill Bit Name into different files, and are reading them in to organize the data.

In [ ]:
BuzzDrill = pd.DataFrame(columns = ['BIT DEPTH', 'RATE OF PENETRATION', 'HOOK LOAD', 'DIFFERENTIAL PRESSURE', 'WEIGHT ON BIT', 'DRILL BIT ID', 'DRILL BIT NAME', 'RUN ID', 'TIME'])
AstroBit = pd.DataFrame(columns = ['BIT DEPTH', 'RATE OF PENETRATION', 'HOOK LOAD', 'DIFFERENTIAL PRESSURE', 'WEIGHT ON BIT', 'DRILL BIT ID', 'DRILL BIT NAME', 'RUN ID', 'TIME'])
ChallengeDriller = pd.DataFrame(columns = ['BIT DEPTH', 'RATE OF PENETRATION', 'HOOK LOAD', 'DIFFERENTIAL PRESSURE', 'WEIGHT ON BIT', 'DRILL BIT ID', 'DRILL BIT NAME', 'RUN ID', 'TIME'])
Apollo = pd.DataFrame(columns = ['BIT DEPTH', 'RATE OF PENETRATION', 'HOOK LOAD', 'DIFFERENTIAL PRESSURE', 'WEIGHT ON BIT', 'DRILL BIT ID', 'DRILL BIT NAME', 'RUN ID', 'TIME'])

In [ ]:
fig = px.scatter(f, x='BIT DEPTH', y='RATE OF PENETRATION', color='DRILL BIT NAME')
fig.show()

Note that the above graph contains a lot of outlier points, which need to be removed. We can do this by grouping the points into intervals of $\Delta x = 500$ and performing a standard deviation outlier check with $m=3$

In [ ]:
OUTLIER_DICT={}
for index, i in enumerate(f['BIT DEPTH']):
    val=i-i%500
    if val in OUTLIER_DICT:
        OUTLIER_DICT[val].append(f['RATE OF PENETRATION'][index])
    else:
        OUTLIER_DICT.update({val:[f['RATE OF PENETRATION'][index]]})
topCutoff = {}
for indice in OUTLIER_DICT:
    topCutoff.update({indice:np.mean(OUTLIER_DICT[indice])+np.std(OUTLIER_DICT[indice])*3})
f['VALID'] = np.nan
for index, element in enumerate(f['BIT DEPTH']):
    f.at[index,'VALID']=f['RATE OF PENETRATION'][index]<topCutoff[element-element%500] 
f=f[f['VALID']]
f=f.drop('VALID', axis=1)

In [ ]:
fig = px.scatter(f, x='BIT DEPTH', y='RATE OF PENETRATION', color='DRILL BIT NAME')
fig.show()

Data points are now much more consistent and will provide better rate of penetration data

In [ ]:
DEPTH_DICT={}
for rate, depth in zip(f['RATE OF PENETRATION'],f['BIT DEPTH']):
    if(depth!=depth):
        continue
    val=depth-depth%50
    if val in DEPTH_DICT:
        DEPTH_DICT[val].append(rate)
    else:
        DEPTH_DICT.update({val:[rate]})
TEMP_X=[]
TEMP_Y=[]
CUR_DEPTH_500_INDEX=150
while CUR_DEPTH_500_INDEX in DEPTH_DICT:
    TEMP_X.append(CUR_DEPTH_500_INDEX)
    TEMP_Y.append(statistics.median(DEPTH_DICT[CUR_DEPTH_500_INDEX]))
    CUR_DEPTH_500_INDEX+=50

In [ ]:
fig= px.scatter(x=TEMP_X, y=TEMP_Y,trendline="lowess",trendline_options=dict(frac=0.08))
fig.show()

The above graph shows the median Rate of Penetration averaged for all drills across Bit Depth. If we assume that all asteroids have some similar density distribution, we can estimate the adjusted speed of each drill to be $\alpha_x =\frac{v_x}{\rho_x}$
Where $\rho$ is the density of the position. Then, we can find $\rho$ by inverting the y-values of this graph

In [ ]:
AVG_HARDNESS_X=fig.data[0]['x']
AVG_HARDNESS_Y=fig.data[0]['y']

In [ ]:
ChallengDrill = pd.read_csv("ChallengDriller.csv")
BuzzDrill = pd.read_csv("Buzz Drilldrin.csv")
AstroDrill = pd.read_csv("Astro Bit.csv")
ApolloDrill = pd.read_csv("Apollo.csv")

In [ ]:
DEPTH_DICT={}
for index, i in enumerate(BuzzDrill['BIT DEPTH']):
    val=i-i%50
    if val in DEPTH_DICT:
        DEPTH_DICT[val].append(BuzzDrill['RATE OF PENETRATION'][index])
    else:
        DEPTH_DICT.update({val:[BuzzDrill['RATE OF PENETRATION'][index]]})
BuzzX=[]
BuzzY=[]
CUR_DEPTH_500_INDEX=min(DEPTH_DICT.keys())
for CUR_DEPTH_500_INDEX in range(300,19050,50):
    if CUR_DEPTH_500_INDEX in DEPTH_DICT:
        BuzzX.append(CUR_DEPTH_500_INDEX)
        BuzzY.append(statistics.median(DEPTH_DICT[CUR_DEPTH_500_INDEX]))

    
df = pd.DataFrame(columns=['Bit Depth', 'Median Rate of Penetration(ft/hr) per 500 ft'])
df['Bit Depth'] = BuzzX
df['Median Rate of Penetration(ft/hr) per 500 ft'] = BuzzY


In [ ]:
DEPTH_DICT={}
for index, i in enumerate(AstroDrill['BIT DEPTH']):
    val=i-i%50
    if val in DEPTH_DICT:
        DEPTH_DICT[val].append(AstroDrill['RATE OF PENETRATION'][index])
    else:
        DEPTH_DICT.update({val:[AstroDrill['RATE OF PENETRATION'][index]]})
AstroX=[]
AstroY=[]
CUR_DEPTH_500_INDEX=min(DEPTH_DICT.keys())
for CUR_DEPTH_500_INDEX in range(300,19050,50):
    if CUR_DEPTH_500_INDEX in DEPTH_DICT:
        AstroX.append(CUR_DEPTH_500_INDEX)
        AstroY.append(statistics.median(DEPTH_DICT[CUR_DEPTH_500_INDEX]))


    
df = pd.DataFrame(columns=['Bit Depth', 'Median Rate of Penetration(ft/hr) per 500 ft'])
df['Bit Depth'] = AstroX
df['Median Rate of Penetration(ft/hr) per 500 ft'] = AstroY


In [ ]:
DEPTH_DICT={}
for index, i in enumerate(ChallengDrill['BIT DEPTH']):
    val=i-i%50
    if val in DEPTH_DICT:
        DEPTH_DICT[val].append(ChallengDrill['RATE OF PENETRATION'][index])
    else:
        DEPTH_DICT.update({val:[ChallengDrill['RATE OF PENETRATION'][index]]})
ChallengX=[]
ChallengY=[]
SampleSize=[]

for CUR_DEPTH_500_INDEX in range(300,19050,50):
    if CUR_DEPTH_500_INDEX in DEPTH_DICT:
        SampleSize.append(len(DEPTH_DICT[CUR_DEPTH_500_INDEX]))
        ChallengX.append(CUR_DEPTH_500_INDEX)
        ChallengY.append(statistics.median(DEPTH_DICT[CUR_DEPTH_500_INDEX]))  


df = pd.DataFrame(columns=['Bit Depth', 'Median Rate of Penetration(ft/hr) per 500 ft','Sample Size'])
df['Bit Depth'] = ChallengX
df['Median Rate of Penetration(ft/hr) per 500 ft'] = ChallengY
df['Sample Size']= SampleSize


In [ ]:
fig = px.scatter(df, x='Bit Depth',y='Median Rate of Penetration(ft/hr) per 500 ft',hover_name='Sample Size' ,trendline="lowess",trendline_options=dict(frac=0.1))
fig.show()

In [ ]:
DEPTH_DICT={}
for index, i in enumerate(ApolloDrill['BIT DEPTH']):
    val=i-i%50
    if val in DEPTH_DICT:
        DEPTH_DICT[val].append(ApolloDrill['RATE OF PENETRATION'][index])
    else:
        DEPTH_DICT.update({val:[ApolloDrill['RATE OF PENETRATION'][index]]})
ApolloX=[]
ApolloY=[]
CUR_DEPTH_500_INDEX=min(DEPTH_DICT.keys())
for CUR_DEPTH_500_INDEX in range(300,19050,50):
    if CUR_DEPTH_500_INDEX in DEPTH_DICT:
        ApolloX.append(CUR_DEPTH_500_INDEX)
        ApolloY.append(statistics.median(DEPTH_DICT[CUR_DEPTH_500_INDEX]))

    
df = pd.DataFrame(columns=['Bit Depth', 'Median Rate of Penetration(ft/hr) per 500 ft'])
df['Bit Depth'] = ApolloX
df['Median Rate of Penetration(ft/hr) per 500 ft'] = ApolloY


In [ ]:
for num in range(len(BuzzX)):
    BuzzY[num]/=AVG_HARDNESS_Y[int(BuzzX[num]/50-6)]
for num in range(len(AstroX)):
    AstroY[num]/=AVG_HARDNESS_Y[int(AstroX[num]/50-6)]
for num in range(len(ChallengX)):
    ChallengY[num]/=AVG_HARDNESS_Y[int(ChallengX[num]/50-6)]
for num in range(len(ApolloX)):
    ApolloY[num]/=AVG_HARDNESS_Y[int(ApolloX[num]/50-6)]

In [ ]:
currFrame = pd.DataFrame(columns = ['Bit Depth', 'RoP Estimate', 'Bit Drill Name'])

allDepths = []
allRoP = []
allNames = []

for d in range(len(BuzzX)):
    allDepths.append(BuzzX[d])
    allRoP.append(BuzzY[d])
    allNames.append("Buzz Drilldrin")
for d in range(len(AstroX)):
    allDepths.append(AstroX[d])
    allRoP.append(AstroY[d])
    allNames.append("Astro Drill")
for d in range(len(ChallengX)):
    allDepths.append(ChallengX[d])
    allRoP.append(ChallengY[d])
    allNames.append("ChallengDriller")
for d in range(len(ApolloX)):
    allDepths.append(ApolloX[d])
    allRoP.append(ApolloY[d])
    allNames.append("Apollo")

currFrame['Bit Depth'] = allDepths
currFrame['RoP Estimate'] = allRoP
currFrame['Bit Drill Name'] = allNames


In [ ]:
file = open("DepthVRoPSmoothed.json", "w")
file.write(currFrame.to_json())
file.close()

In [ ]:
fig = px.scatter(currFrame, x='Bit Depth', y='RoP Estimate', color='Bit Drill Name',trendline="lowess",trendline_options=dict(frac=0.1))
fig.data = [t for t in fig.data if t.mode == "lines"]
fig.update_traces(showlegend=True)
fig.show()

In [ ]:
TestFrame = pd.DataFrame(columns = ['Bit Depth', 'RoP Estimate', 'Bit Drill Name'])

titles = ["Buzz Drilldrin", "Astro Drill", "ChallengDriller", "Apollo"]

temp_RoP=[]
temp_titles=[]
for j in range(4):
    temp_RoP+=[i*AVG_HARDNESS_Y[index] for index,i in enumerate(fig.data[j]['y'])]
    temp_titles+=[titles[j]]*len(fig.data[j]['y'])
TestFrame['Bit Depth'] = currFrame['Bit Depth']
TestFrame['RoP Estimate'] = [1/i for i in temp_RoP]
TestFrame['Bit Drill Name'] = temp_titles

In [ ]:
file = open("InvertedDepthByRoP.json", "w")
file.write(TestFrame.to_json())
file.close()

In [ ]:
fig = px.scatter(TestFrame, x='Bit Depth',y='RoP Estimate', color='Bit Drill Name',trendline="lowess",trendline_options=dict(frac=0.08))
fig.data = [t for t in fig.data if t.mode == "lines"]
fig.update_traces(showlegend=True)
fig.show()

In [ ]:
SUM=[0.0,0.0,0.0,0.0]
RATE=[[],[],[],[]]
HOURS_PER_FOOT=[]
for i in range(4):
    HOURS_PER_FOOT.append(fig.data[i]['y'])
    for j in fig.data[i]['y']:
        RATE[i].append(SUM[i])
        SUM[i]+=(j)*50

In [ ]:
IntegratedFrame = pd.DataFrame(columns = ['Bit Depth', 'Hours per Foot', 'Bit Drill Name'])

titles = ["Buzz Drilldrin", "Astro Drill", "ChallengDriller", "Apollo"]
id_from_drill_name ={"Buzz Drilldrin":0, "Astro Drill":1, "ChallengDriller":2, "Apollo":3}

names = []
rops = []

for i in range(4):
    for j in range(len(RATE[i])):
        names.append(titles[i])
        rops.append(RATE[i][j])
IntegratedFrame['Bit Depth'] = currFrame['Bit Depth']
IntegratedFrame['Hours per Foot'] = rops
IntegratedFrame['Bit Drill Name'] = names

In [ ]:
file = open("IntegratedDF.json", "w")
file.write(IntegratedFrame.to_json())
file.close()

In [ ]:
fig = px.line(IntegratedFrame, x='Bit Depth',y='Hours per Foot', color='Bit Drill Name')
fig.show()

In [ ]:
totalHours=0.0
Hour_per_foot=[]
final_depth=[]
final_color=[]
switch_dict = {7800:"ChallengDriller", 14450:"Apollo"}
curDrill="Astro Drill"
for index, i in enumerate(range(350,19050,50)):
    if(i in switch_dict):
        curDrill = switch_dict[i]
        totalHours+=i/12000
    final_color.append(curDrill)
    totalHours+=HOURS_PER_FOOT[id_from_drill_name[curDrill]][index]*50
    Hour_per_foot.append(totalHours)
    final_depth.append(i)

In [ ]:
fig= px.line(y=Hour_per_foot, x=final_depth,color=final_color)
fig.show()